In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import os, sys
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split, KFold
from sklearn.linear_model import Ridge
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.decomposition import RandomizedPCA

from collections import defaultdict

from scipy.optimize import nnls

sns.set_style('whitegrid')
sns.set_context('poster')

import warnings
warnings.filterwarnings('ignore')

basepath = os.path.expanduser('~/Desktop/src/African_Soil_Property_Prediction/')
sys.path.append(os.path.join(basepath, 'src'))

np.random.seed(0)

from data import make_dataset
from models import eval_metric

In [2]:
# load files
train = pd.read_csv(os.path.join(basepath, 'data/raw/training.csv'))
test = pd.read_csv(os.path.join(basepath, 'data/raw/sorted_test.csv'))
sample_sub = pd.read_csv(os.path.join(basepath, 'data/raw/sample_submission.csv'))

** Group feature by spectral band. **

In [3]:
spectral_band = re.compile(r'([a-z]+)([0-9]+)')

def group_by_wavelength(column_names):
    band_dict = defaultdict(list)
    
    for col in column_names:
        match = spectral_band.match(col)

        alpha, numeric = match.groups()
        n = len(numeric)

        band_dict[int(numeric[0]) * (10 ** (n - 1))].append(col)
    
    return band_dict

band_dict = group_by_wavelength(train.columns[1:-21])

In [4]:
def create_df(df, band_dict):
    new_df = {}
    
    for k, v in band_dict.items():
        new_df[k] = df[v].mean(axis=1)
    
    return pd.DataFrame(new_df)

train_ = create_df(train, band_dict)
test_ = create_df(test, band_dict)

In [5]:
print('==== Summary statistics for training examples')
train_.describe()

==== Summary statistics for training examples


,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000
count,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000,1157.000000
mean,1.566095,1.636803,1.564623,1.519612,1.416589,1.334178,0.701871,1.013991,0.330380,0.277733,0.260518,0.250557
std,0.171090,0.232786,0.264389,0.198303,0.222034,0.107357,0.152323,0.228178,0.115951,0.114500,0.113424,0.113685
min,1.023430,0.916383,0.897768,1.008594,0.768420,0.982845,0.317568,0.372794,0.044316,0.004844,-0.015332,-0.039193
25%,1.443850,1.485915,1.374557,1.367704,1.264880,1.266322,0.587672,0.844588,0.252499,0.199189,0.182272,0.174341
50%,1.565840,1.669190,1.584972,1.535751,1.435184,1.350239,0.712711,1.058943,0.341380,0.283231,0.266833,0.259560
75%,1.687500,1.802365,1.758571,1.658422,1.568500,1.407097,0.819660,1.198105,0.402424,0.344373,0.326784,0.320106
max,2.014480,2.214068,2.261778,2.083415,1.995202,1.576287,1.062781,1.448074,0.774800,0.776815,0.760040,0.735014


In [6]:
print('==== Summary statistics for test examples')
test_.describe()

==== Summary statistics for test examples


,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000
count,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000
mean,1.575379,1.667252,1.591736,1.537806,1.448867,1.329456,0.703868,1.036910,0.326016,0.269047,0.249911,0.240069
std,0.156563,0.191165,0.209014,0.164417,0.180780,0.093227,0.121393,0.187894,0.095378,0.101224,0.099919,0.097544
min,1.028730,1.209142,1.128883,1.030293,0.990573,0.998014,0.351262,0.465867,0.077545,0.005339,-0.014956,-0.025691
25%,1.477760,1.522999,1.424173,1.422708,1.309847,1.274846,0.617776,0.909537,0.253147,0.205325,0.187089,0.177324
50%,1.583240,1.692839,1.607334,1.548303,1.464973,1.337033,0.710981,1.062979,0.339193,0.282206,0.263772,0.256387
75%,1.693490,1.805904,1.745914,1.656638,1.576325,1.392217,0.793980,1.175343,0.392073,0.336428,0.314247,0.302366
max,1.915400,2.198115,2.260722,1.979957,1.981086,1.563511,1.011522,1.457121,0.575990,0.542214,0.529400,0.502016


** Prepare dataset. **

In [50]:
X = train_.copy()
Xtest = test_.copy()

y_Ca = train.Ca
y_SOC = train.SOC
y_Sand = train.Sand
y_pH = train.pH
y_P = train.P

** Split dataset. **

In [51]:
def split_dataset(train_length, **params):
    itrain, itest = train_test_split(range(train_length), **params)
    
    return itrain, itest

In [52]:
params = {
    'test_size': 0.2,
    'random_state': 3
}

itrain, itest = split_dataset(len(X), **params)

X_train = X.iloc[itrain]
X_test = X.iloc[itest]

y_train_Ca = y_Ca.iloc[itrain]
y_test_Ca = y_Ca.iloc[itest]

y_train_P = y_P.iloc[itrain]
y_test_P = y_P.iloc[itest]

y_train_Sand = y_Sand.iloc[itrain]
y_test_Sand = y_Sand.iloc[itest]

y_train_SOC = y_SOC.iloc[itrain]
y_test_SOC = y_SOC.iloc[itest]

y_train_pH = y_pH.iloc[itrain]
y_test_pH = y_pH.iloc[itest]

In [83]:
pipeline1 = Pipeline([
        ('scale', StandardScaler()),
        ('model', SVR(C=10., gamma=.1))
    ])

pipeline2 = Pipeline([
        ('scale', StandardScaler()),
        ('model', SVR(C=10., gamma=.1))
    ])

pipeline3 = Pipeline([
        ('scale', StandardScaler()),
        ('model', SVR(C=10., gamma=.1))
    ])

pipeline4 = Pipeline([
        ('scale', StandardScaler()),
        ('model', SVR(C=10., gamma=.1))
    ])

pipeline5 = Pipeline([
        ('scale', StandardScaler()),
        ('model', SVR(C=10., gamma=.1))
    ])

** Set up cross validation scheme. **

In [95]:
def cv_scheme(pipelines, X, y_Ca, y_P, y_Sand, y_SOC, y_pH):
    cv = KFold(len(X), n_folds=5, shuffle=True, random_state=10)
    
    scores = 0
    for itrain, itest in cv:
        Xtr = X.iloc[itrain]
        
        ytr_Ca = y_Ca.iloc[itrain]
        ytr_P = y_P.iloc[itrain]
        ytr_Sand = y_Sand.iloc[itrain]
        ytr_SOC = y_SOC.iloc[itrain]
        ytr_pH = y_pH.iloc[itrain]
        
        Xte = X.iloc[itest]
        
        yte_Ca = y_Ca.iloc[itest]
        yte_P = y_P.iloc[itest]
        yte_Sand = y_Sand.iloc[itest]
        yte_SOC = y_SOC.iloc[itest]
        yte_pH = y_pH.iloc[itest]
    
        pipelines[0].fit(Xtr, ytr_Ca)
        pipelines[1].fit(Xtr, ytr_P)
        pipelines[2].fit(Xtr, ytr_Sand)
        pipelines[3].fit(Xtr, ytr_SOC)
        pipelines[4].fit(Xtr, ytr_pH)
        
        ypred_Ca = pipelines[0].predict(Xte)
        ypred_P = pipelines[1].predict(Xte)
        ypred_Sand = pipelines[2].predict(Xte)
        ypred_SOC = pipelines[3].predict(Xte)
        ypred_pH = pipelines[4].predict(Xte)

        scores += eval_metric.mcrmse([yte_Ca, yte_P, yte_pH, yte_Sand, yte_SOC], [ypred_Ca, ypred_P, ypred_pH, ypred_Sand, ypred_SOC])
    
    return scores / len(cv)

In [96]:
scores = cv_scheme([
        pipeline1,
        pipeline2,
        pipeline3,
        pipeline4,
        pipeline5
    ], X_train, y_train_Ca, y_train_P, y_train_Sand, y_train_SOC, y_train_pH)

In [97]:
print('Score after 5-fold cross-validation: %f'%scores)

Score after 5-fold cross-validation: 0.626360


In [84]:
pipeline1.fit(X_train, y_train_Ca)
pipeline2.fit(X_train, y_train_P)
pipeline3.fit(X_train, y_train_Sand)
pipeline4.fit(X_train, y_train_SOC)
pipeline5.fit(X_train, y_train_pH)

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])

In [85]:
y_pred_Ca = (pipeline1.predict(X_test))
y_pred_P = (pipeline2.predict(X_test))
y_pred_Sand = (pipeline3.predict(X_test))
y_pred_SOC = (pipeline4.predict(X_test))
y_pred_pH = (pipeline5.predict(X_test))

** Private Leaderboard Score: 0.71622 **

In [86]:
print('MCRMSE on unseen examples: %f'%eval_metric.mcrmse([y_test_Ca, y_test_P, y_test_pH, y_test_Sand, y_test_SOC], [y_pred_Ca, y_pred_P, y_pred_pH, y_pred_Sand, y_pred_SOC]))

MCRMSE on unseen examples: 0.617533


** Training. **

In [87]:
pipeline1.fit(X, y_Ca)
pipeline2.fit(X, y_P)
pipeline3.fit(X, y_Sand)
pipeline4.fit(X, y_SOC)
pipeline5.fit(X, y_pH)

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])

In [88]:
predict_Ca = pipeline1.predict(Xtest) 
predict_P = pipeline2.predict(Xtest)
predict_Sand = pipeline3.predict(Xtest)
predict_SOC = pipeline4.predict(Xtest)
predict_pH = pipeline5.predict(Xtest)

** Submission. **

In [89]:
sample_sub['Ca'] = predict_Ca
sample_sub['P'] = predict_P
sample_sub['pH'] = predict_pH
sample_sub['SOC'] = predict_SOC
sample_sub['Sand'] = predict_Sand

In [90]:
sample_sub.to_csv(os.path.join(basepath, 'submissions/group_by_band.csv'), index=False)